In [0]:
# STEP 1

# #This section is base copy from google's sample import of tensorflow, keras, ploting and support libraries

#keras install libraries
!pip install -q keras
!pip install -q tqdm

#openCV is used to process and filter images
!apt-get -qq install -y libsm6 libxext6 && pip install -q -U opencv-python

In [3]:
# STEP 2
# In this step we import the libraries needed to import files from google drive

# Install a Drive FUSE wrapper.
# https://github.com/astrada/google-drive-ocamlfuse
!apt-get install -y -qq software-properties-common python-software-properties module-init-tools
!add-apt-repository -y ppa:alessandro-strada/ppa 2>&1 > /dev/null
!apt-get update -qq 2>&1 > /dev/null
!apt-get -y install -qq google-drive-ocamlfuse fuse

E: Package 'python-software-properties' has no installation candidate
Selecting previously unselected package google-drive-ocamlfuse.
(Reading database ... 145113 files and directories currently installed.)
Preparing to unpack .../google-drive-ocamlfuse_0.7.17-0ubuntu1~ubuntu18.04.1_amd64.deb ...
Unpacking google-drive-ocamlfuse (0.7.17-0ubuntu1~ubuntu18.04.1) ...
Setting up google-drive-ocamlfuse (0.7.17-0ubuntu1~ubuntu18.04.1) ...
Processing triggers for man-db (2.8.3-2ubuntu0.1) ...


In [0]:
# STEP 3
# Generate auth tokens for Colab

# CLICK THE LINK GENERATED THEN COPY AND PASTE THE KEY INTO THE FIELD AND CLICK ENTER
from google.colab import auth
auth.authenticate_user()

In [0]:
# STEP 4
# Here we connect to your google drive folder.
# You can download sample puppy and rabbit images using this link: https://drive.google.com/drive/folders/1RcWFv-PDtGK4n1JgkAgL77fB-N0I4oMP?usp=sharing

# WARNING YOU MAY NEED TO RUN THIS CELL MULTIPLE TIMES TO GET A CONNECTION + KEY ESTABLISHED to your google drive.

# CLICK THE 1st LINK GENERATED THEN COPY AND PASTE THE KEY INTO THE FIELD AND CLICK ENTER

# RE-RUN THIS CELL AND CLICK THE 2nd LINK GENERATED THEN COPY AND PASTE THE KEY INTO THE FIELD AND CLICK ENTER

# Generate creds for the Drive FUSE library.
from oauth2client.client import GoogleCredentials
creds = GoogleCredentials.get_application_default()
import getpass
!google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret} < /dev/null 2>&1 | grep URL
!echo {vcode} | google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret}
vcode = getpass.getpass()



In [7]:
# STEP 5
# Create a directory and mount Google Drive using that directory.
!mkdir -p drive
!google-drive-ocamlfuse drive
# print 'Files in Drive:'
!ls drive/

 Classification   CnnModel.ipynb  'Colab Notebooks'  'Getting started'


In [0]:
import sys
import os
from keras.preprocessing.image import ImageDataGenerator
from keras import optimizers
from keras.models import Sequential
from keras.layers import Dropout, Flatten, Dense, Activation
from keras.layers.convolutional import Convolution2D, MaxPooling2D
from keras import callbacks
epochs = 20
train_data_path = 'drive/Classification/train/'
validation_data_path = 'drive/Classification/test/'

"""
Parameters
"""
img_width, img_height = 64, 64
batch_size = 32
samples_per_epoch = 4000
validation_steps = 400
nb_filters1 = 32
nb_filters2 = 64
conv1_size = 3
conv2_size = 2
pool_size = 2
classes_num = 3
lr = 0.001

model = Sequential()
model.add(Convolution2D(nb_filters1, conv1_size, conv1_size, border_mode ="same", input_shape=(img_width, img_height, 3)))
model.add(Activation("relu"))
model.add(MaxPooling2D(pool_size=(pool_size, pool_size)))

model.add(Convolution2D(nb_filters2, conv2_size, conv2_size, border_mode ="same"))
model.add(Activation("relu"))
model.add(MaxPooling2D(pool_size=(pool_size, pool_size), dim_ordering='th'))


model.add(Convolution2D(nb_filters2, conv2_size, conv2_size, border_mode ="same"))
model.add(Activation("relu"))
model.add(MaxPooling2D(pool_size=(pool_size, pool_size), dim_ordering='th'))


model.add(Flatten())
model.add(Dense(256))
model.add(Activation("relu"))
model.add(Dropout(0.5))
model.add(Dense(classes_num, activation='softmax'))

model.compile(loss='categorical_crossentropy',
              optimizer=optimizers.RMSprop(lr=lr),
              metrics=['accuracy'])

train_datagen = ImageDataGenerator(
    rescale=1. / 255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True)

test_datagen = ImageDataGenerator(rescale=1. / 255)

train_generator = train_datagen.flow_from_directory(
    train_data_path,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='categorical')

validation_generator = test_datagen.flow_from_directory(
    validation_data_path,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='categorical')

 

model.fit_generator(
    train_generator,
    samples_per_epoch=samples_per_epoch,
    epochs=epochs,
    validation_data=validation_generator,
  
    validation_steps=validation_steps)

#target_dir = 'drive/Image Processing DataSet/'
#if not os.path.exists(target_dir):
 # os.mkdir(target_dir)
#model.save('drive/Image Processing DataSet/model.h5')
#model.save_weights('drive/Image Processing DataSet/weights.h5')


Using TensorFlow backend.


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:29: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(32, (3, 3), input_shape=(64, 64, 3..., padding="same")`
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:33: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(64, (2, 2), padding="same")`
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:35: UserWarning: Update your `MaxPooling2D` call to the Keras 2 API: `MaxPooling2D(pool_size=(2, 2), data_format="channels_first")`






Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.




/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:38: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(64, (2, 2), padding="same")`
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:40: UserWarning: Update your `MaxPooling2D` call to the Keras 2 API: `MaxPooling2D(pool_size=(2, 2), data_format="channels_first")`


Found 767 images belonging to 3 classes.
Found 192 images belonging to 3 classes.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where



/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:81: UserWarning: Update your `fit_generator` call to the Keras 2 API: `fit_generator(<keras_pre..., epochs=20, validation_data=<keras_pre..., validation_steps=400, steps_per_epoch=125)`



Epoch 1/20
 12/125 [=>............................] - ETA: 26:29 - loss: 1.3181 - acc: 0.3984

In [0]:

import os
import cv2 
import numpy as np
from keras.preprocessing.image import ImageDataGenerator, load_img, img_to_array
from keras.models import Sequential, load_model
import matplotlib.pyplot as plt # for visualizations
import matplotlib.image as mpimg
from PIL import Image
img_width, img_height = 64, 64


def predict(file):
  x = load_img(file, target_size=(img_width,img_height))
  x = img_to_array(x)
  x = np.expand_dims(x, axis=0)
  array = model.predict(x)
  result = array[0]
  answer = np.argmax(result)
  label = ''
  if answer == 0:
    print("Label: Arslan")
    label= 'Arslan'
  elif answer == 1:
    print("Label: Fasih")
    label= 'Fasih'
    #cv2.imshow(file)
  elif answer == 2:
    label = 'Mehraj'
    print("Label: Mehraj")
    #cv2.imshow(file)
  img = mpimg.imread(file)
    
  plt.imshow(img)
  plt.title("Pred:  "+label )
  plt.axis('off')
  plt.show()
  return answer

arsal_t = 0
arsal_f = 0
fas_t = 0
fas_f = 0
meh_t = 0
meh_f = 0

for i, ret in enumerate(os.walk('drive/Classification/new/')):
  for i, filename in enumerate(ret[2]):
    if filename.startswith("."):
      continue
    print("Label:data_A " +' Orignal '+ filename)
    result = predict(ret[0] + '/' + filename)
    if result == 0:
      arsal_t += 1
    else:
      arsal_f += 1

for i, ret in enumerate(os.walk('drive/Classification/new/')):
  for i, filename in enumerate(ret[2]):
    if filename.startswith("."):
      continue
    print("Label:data_F")
    result = predict(ret[0] + '/' + filename)
    
    if result == 1:
      fas_t += 1
    else:
      fas_f += 1

for i, ret in enumerate(os.walk('drive/Classification/new/')):
  for i, filename in enumerate(ret[2]):
    if filename.startswith("."):
      continue
    print("Label: data_M")
    result = predict(ret[0] + '/' + filename)
    if result == 2:
      meh_t += 1
    else:
      meh_f += 1

"""
Check metrics
"""
print("True 1: ", arsal_t)
print("False 1: ", arsal_f)
print("True 2: ", fas_t)
print("False 2: ", fas_f)
print("True 3: ", meh_t)
print("False 3: ", meh_f)

Using TensorFlow backend.


True 1:  0
False 1:  0
True 2:  0
False 2:  0
True 3:  0
False 3:  0
